# Clean Used Car Data

In [1]:
import pandas as pd 
import numpy as np 
import statistics as stat 
import sys
sys.path.append(".")
import logging as logger

## Load the data & check the data types

In [2]:
#load data into dataframe for cleaning 

raw_data = pd.read_csv("../data/vehicles.csv")
  

raw_data.shape # checking the expected data has arrived

In [ ]:
raw_data.head(10)

In [30]:
%%time
raw_data.info() # get the information breakdown of the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

## Initial Notes
1. The data shows a number of features that contain null values.
2. There are a number of features that are not required for modelling

## Dropping columns no required 

In [4]:
#drop all columns not required for modelling 
data_1 = raw_data.drop(columns=['id', 'url','region_url','VIN', 'county', 'lat','long', 'posting_date', 'image_url', 'description', 'model'])

## Calulate the number of NaN's per columns

In [5]:
#count up the number of nulls 

null_check = data_1.isnull().sum()
null_check = pd.DataFrame(null_check, columns=["Null"])
total = len(raw_data)
null_check['%'] = pd.DataFrame(null_check['Null']/total)
null_check.sort_values('%',ascending=False)


,Null,%
size,306361,0.717675
cylinders,177678,0.416225
condition,174104,0.407852
drive,130567,0.305863
paint_color,130203,0.305011
type,92858,0.217527
manufacturer,17646,0.041337
title_status,8242,0.019308
odometer,4400,0.010307
fuel,3013,0.007058


## Removing small levels of rows that contain NaN's 

In [6]:
#removing the rows that are missing <= 10% of the over data 
data_2 = data_1.dropna(subset=['year','transmission', 'fuel', 'odometer','title_status'])
null_check_2 = data_2.isnull().sum()
null_check_2 = pd.DataFrame(null_check_2, columns=["Null"])
total = len(raw_data)
null_check_2['%'] = pd.DataFrame(null_check_2['Null']/total)
null_check_2.sort_values('%',ascending=False)

,Null,%
size,291457,0.682761
cylinders,171613,0.402017
condition,165412,0.387491
drive,124179,0.290899
paint_color,121109,0.283707
type,91027,0.213238
manufacturer,15991,0.037460
region,0,0.000000
price,0,0.000000
year,0,0.000000


# Check the value counts for all the categorical features

In [7]:
# check the catigorical type of data
data_2['size'].value_counts()

full-size      62884
mid-size       34332
compact        19018
sub-compact     3051
Name: size, dtype: int64

In [8]:
data_2['condition'].value_counts()

good         121121
excellent     94362
like new      21178
fair           6769
new            1299
salvage         601
Name: condition, dtype: int64

In [9]:
data_2['drive'].value_counts()

4wd    126624
fwd    102445
rwd     57494
Name: drive, dtype: int64

In [10]:
data_2['paint_color'].value_counts()

white     77258
black     61283
silver    41966
blue      30576
red       29842
grey      23770
green      7253
custom     6507
brown      6498
yellow     2068
orange     1935
purple      677
Name: paint_color, dtype: int64

In [11]:
data_2['type'].value_counts()

sedan          83359
SUV            73062
pickup         41961
truck          34142
other          20885
coupe          18653
hatchback      16199
wagon          10312
van             7775
convertible     7529
mini-van        4712
offroad          609
bus              517
Name: type, dtype: int64

In [12]:
data_2['manufacturer'].value_counts()

ford               68527
chevrolet          53239
toyota             32767
honda              20521
jeep               18290
nissan             18170
ram                17722
gmc                16150
bmw                14339
dodge              13130
mercedes-benz      11316
hyundai             9931
subaru              9197
volkswagen          9111
kia                 8162
lexus               7957
audi                7363
cadillac            6628
acura               5837
chrysler            5749
buick               5345
mazda               5229
infiniti            4708
lincoln             4065
volvo               3317
mitsubishi          3111
mini                2302
pontiac             2235
rover               2028
jaguar              1917
porsche             1327
mercury             1164
saturn              1064
alfa-romeo           887
tesla                834
fiat                 773
harley-davidson      135
ferrari               93
datsun                63
aston-martin          24


In [45]:
data_2['title_status'].value_counts()

clean         397362
rebuilt         7083
salvage         3863
lien            1422
missing          814
parts only       198
Name: title_status, dtype: int64

In [ ]:
data_2['region'].value_counts()

In [ ]:
data_2['fuel'].value_counts()

In [ ]:
data_2['transmission'].value_counts()

In [ ]:
data_2['state'].value_counts()

## Notes 
1. fuel has 30k records  classed as other. Unable to know or predict the fuel type so these row will be removed
2. Transmission as ~62k records classed as other. Unable to know or predict the transamission type so these rows will be removed. 
3. in the feature title status, the field is used to highlight if the item being sold is 

In [13]:
#removing the rows of data as per the notes above 
data_3 = data_2[data_2['fuel'] != 'other']
data_3 = data_2[data_2['transmission'] != 'other']

In [14]:
null_check_3 = data_3.isnull().sum()
null_check_3 = pd.DataFrame(null_check_3, columns=["Null"])
total = len(raw_data)
null_check_3['%'] = pd.DataFrame(null_check_3['Null']/total)
null_check_3.sort_values('%',ascending=False)


,Null,%
size,230811,0.540693
condition,161632,0.378636
cylinders,135622,0.317705
paint_color,108778,0.254821
drive,97932,0.229413
type,89538,0.209750
manufacturer,13570,0.031789
region,0,0.000000
price,0,0.000000
year,0,0.000000


## Encoding Condition features
Preparing each of the features for imputation using KNN.
Each feature will require its NaNs rows removed to a seperate dataframe. These will be merged back once the feature is encoded. 


In [15]:
## seperated the data into two dataframes. One contain all rows with no nan and the other rows with nans
prep_encoding = data_3
nan_data = data_3[data_3.isna().any(axis=1)]

In [16]:
prep_encoding

,region,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
31,auburn,15000,2013.0,ford,excellent,6 cylinders,gas,128000.0,clean,automatic,rwd,full-size,truck,black,al
34,auburn,35000,2019.0,toyota,excellent,6 cylinders,gas,43000.0,clean,automatic,4wd,NaN,truck,grey,al
37,auburn,4500,1992.0,jeep,excellent,6 cylinders,gas,192000.0,clean,automatic,4wd,NaN,NaN,NaN,al
46,auburn,0,2011.0,jeep,excellent,NaN,gas,99615.0,clean,automatic,NaN,full-size,SUV,NaN,al
55,auburn,19900,2004.0,ford,good,8 cylinders,diesel,88000.0,clean,automatic,4wd,full-size,pickup,blue,al
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426864,wyoming,29590,2016.0,toyota,good,6 cylinders,gas,53475.0,clean,automatic,NaN,NaN,SUV,black,wy
426866,wyoming,32990,2016.0,infiniti,good,8 cylinders,gas,55612.0,clean,automatic,rwd,NaN,other,black,wy
426868,wyoming,0,2010.0,toyota,excellent,6 cylinders,gas,155000.0,clean,automatic,4wd,NaN,NaN,blue,wy
426873,wyoming,30990,2018.0,mercedes-benz,good,NaN,gas,15080.0,clean,automatic,rwd,NaN,other,white,wy


In [ ]:
prep_encoding
endcoded = prep_encoding

In [17]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [18]:

#col = prep_encoding.columns
encoder_dict = {}
for name in prep_encoding:
    #encoder for the column
    encoder_dict[name] = OrdinalEncoder()
    
    #select no nulls 
    col = prep_encoding[name]
    col_notnull = col[col.notnull()]
    reshape = col_notnull.values.reshape(-1,1)

    #encode non-nills 
    encoded_vals = encoder_dict[name].fit_transform(reshape)
    prep_encoding.loc[col.notnull(), name] = np.squeeze(encoded_vals)

/Users/marcofoley/opt/anaconda3/envs/NCI_1/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/marcofoley/opt/anaconda3/envs/NCI_1/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/marcofoley/opt/anaconda3/envs/NCI_1/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarn

In [19]:
prep_encoding

,region,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
31,16.0,4977.0,102.0,13.0,0.0,5.0,2.0,65613.0,0.0,0.0,2.0,1.0,10.0,0.0,1.0
34,16.0,11171.0,108.0,39.0,0.0,5.0,2.0,20084.0,0.0,0.0,0.0,NaN,10.0,5.0,1.0
37,16.0,1287.0,81.0,20.0,0.0,5.0,2.0,88672.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0
46,16.0,0.0,100.0,20.0,0.0,NaN,2.0,50272.0,0.0,0.0,NaN,1.0,0.0,NaN,1.0
55,16.0,6932.0,93.0,13.0,2.0,6.0,0.0,43526.0,0.0,0.0,0.0,1.0,8.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426864,397.0,9858.0,105.0,39.0,2.0,5.0,2.0,25344.0,0.0,0.0,NaN,NaN,0.0,0.0,50.0
426866,397.0,10694.0,105.0,18.0,2.0,6.0,2.0,26377.0,0.0,0.0,2.0,NaN,7.0,0.0,50.0
426868,397.0,0.0,99.0,39.0,0.0,5.0,2.0,77722.0,0.0,0.0,0.0,NaN,NaN,1.0,50.0
426873,397.0,10235.0,107.0,26.0,2.0,NaN,2.0,5419.0,0.0,0.0,2.0,NaN,7.0,10.0,50.0


# impute the NANs using KNN 

In [25]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [26]:
imputer = KNNImputer(n_neighbors=5)
scale = MinMaxScaler()
scaled_data = pd.DataFrame(scale.fit_transform(prep_encoding), columns=prep_encoding.columns)



In [32]:
%%time
sample = scaled_data.sample(frac=0.1, replace=False, random_state=1)
finaldataset = pd.DataFrame(imputer.fit_transform(sample), columns=scaled_data.columns)


CPU times: user 1min 39s, sys: 22.9 s, total: 2min 2s
Wall time: 1min 28s


In [48]:
x

,region,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,122.0,1852.0,97.0,16.0,0.6,4.8,2.0,69560.0,0.0,0.0,1.0,1.0,9.0,9.0,9.0
1,27.0,10469.0,108.0,37.0,1.2,3.0,2.0,8104.0,0.0,0.0,0.0,1.4,0.0,7.2,26.0
2,80.0,5363.0,96.0,6.0,1.2,6.0,2.0,61192.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0
3,61.0,469.0,100.0,20.0,0.0,4.4,2.0,39198.0,0.0,0.0,0.4,1.8,0.6,3.8,35.0
4,399.0,1767.0,99.0,39.0,0.0,3.8,2.0,71655.0,0.0,0.0,1.0,0.6,9.0,8.0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34867,232.0,2790.0,103.0,8.0,0.0,5.0,2.0,42332.0,0.0,0.0,1.0,1.4,7.6,1.0,6.0
34868,340.0,11447.0,107.0,7.0,0.0,6.0,2.0,23426.0,0.0,0.0,0.0,1.0,10.0,5.0,13.0
34869,189.0,1308.0,93.0,7.0,2.0,5.0,2.0,87744.0,0.0,0.0,0.0,1.0,0.0,9.0,38.0
34870,131.0,1134.0,96.0,39.0,2.0,3.0,3.0,88649.0,0.0,0.0,1.0,1.0,9.0,9.0,27.0


In [41]:
x = pd.DataFrame(scale.inverse_transform(finaldataset), columns=finaldataset.columns)

In [55]:
for col in x:
    #encoder for the column
   
    reshape_col = x[col].values.reshape(-1,1)
    x[col] = encoder_dict[col].inverse_transform(reshape_col)

In [57]:
x.isnull().any()

region          False
price           False
year            False
manufacturer    False
condition       False
cylinders       False
fuel            False
odometer        False
title_status    False
transmission    False
drive           False
size            False
type            False
paint_color     False
state           False
dtype: bool

In [61]:
x.head(35)

,region,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,gainesville,6650,2008.0,honda,excellent,5 cylinders,gas,136146.0,clean,automatic,fwd,full-size,sedan,silver,fl
1,bend,31998,2019.0,subaru,fair,4 cylinders,gas,20809.0,clean,automatic,4wd,full-size,SUV,purple,mt
2,denver,15998,2007.0,cadillac,fair,8 cylinders,gas,119408.0,clean,automatic,4wd,full-size,SUV,black,co
3,cleveland,500,2011.0,jeep,excellent,5 cylinders,gas,80125.0,clean,automatic,4wd,full-size,SUV,custom,oh
4,york,6400,2010.0,toyota,excellent,4 cylinders,gas,140629.0,clean,automatic,fwd,compact,sedan,red,pa
5,la crosse,25500,2018.0,kia,like new,4 cylinders,gas,56500.0,clean,automatic,4wd,full-size,hatchback,black,wi
6,charlottesville,29988,2007.0,gmc,excellent,6 cylinders,diesel,163063.0,clean,automatic,rwd,full-size,truck,white,tn
7,missoula,23980,2008.0,cadillac,fair,8 cylinders,gas,49012.0,clean,automatic,4wd,full-size,SUV,black,mt
8,des moines,18995,2011.0,ram,good,8 cylinders,gas,123825.0,clean,automatic,4wd,full-size,truck,blue,ia
9,south florida,7900,2015.0,nissan,excellent,4 cylinders,gas,109000.0,clean,automatic,fwd,compact,other,orange,fl


In [ ]:
#Ordinal Encoders
sizeorder = ['sub-compact', 'compact', 'mid-size', 'full-size']
size_encoder = OrdinalEncoder(categories =[sizeorder])
conditionorder = ['salvage', 'fair','good','like new', 'new', 'excellent']
condition_encoder = OrdinalEncoder(categories=[conditionorder])
titleorder = ['parts only', 'missing','lien','salvage', 'rebuilt', 'clean']
title_encoder = OrdinalEncoder(categories=[titleorder])

# Label encoders
region_encoder = LabelEncoder()
manuf_encoder = LabelEncoder()
cylinder_encoder = LabelEncoder()
paint_encoder = LabelEncoder()
type_encoder = LabelEncoder()
drive_encoder = LabelEncoder()
transmission_encoder = LabelEncoder()
state_encoder = LabelEncoder()
fuel_encoder = LabelEncoder()




In [ ]:
#Fit the encoders with data
size_encoder.fit(prep_encoding[['size']])
size_labels = size_encoder.fit_transform(prep_encoding[['size']])

# condition_encoder.fit(prep_encoding[['condition']])
# condition_labels = condition_encoder.fit_transform(prep_encoding[['condition']])

# region_encoder.fit(prep_encoding[['region']])
# region_labels = region_encoder.fit_transform(prep_encoding[['region']])

# manuf_encoder.fit(prep_encoding[['manufacturer']])
# manuf_labels = manuf_encoder.fit_transform(prep_encoding[['manufacturer']])

# cylinder_encoder.fit(prep_encoding[['cylinders']])
# cylinder_labels = cylinder_encoder.fit_transform(prep_encoding[['cylinders']]) 

# paint_encoder.fit(prep_encoding[['paint_color']])
# paint_labels = paint_encoder.fit_transform(prep_encoding[['paint_color']])

# type_encoder.fit(prep_encoding[['type']])
# type_labels = type_encoder.fit_transform(prep_encoding[['type']])

# drive_encoder.fit(prep_encoding[['drive']])
# drive_labels = drive_encoder.fit_transform(prep_encoding[['drive']])

# transmission_encoder.fit(prep_encoding[['transmission']])
# transmission_labels = transmission_encoder.fit_transform(prep_encoding[['transmission']])

# state_encoder.fit(prep_encoding[['state']])
# state_labels = state_encoder.fit_transform(prep_encoding[['state']])

# fuel_encoder.fit(prep_encoding[['fuel']])
# fuel_labels = fuel_encoder.fit_transform(prep_encoding[['fuel']])

# title_encoder.fit(prep_encoding[['title_status']])
# title_labels = title_encoder.fit_transform(prep_encoding[['title_status']])

In [ ]:
#Adding Labels back into the data set 
endcoded['size'] = size_labels
endcoded['condition'] = condition_labels
endcoded['region'] = region_labels
endcoded['manufacturer'] = manuf_labels
endcoded['cylinders'] = cylinder_labels
endcoded['paint_color'] = paint_labels
endcoded['type'] = type_labels
endcoded['drive'] = drive_labels
endcoded['transmission'] = transmission_labels
endcoded['state'] = state_labels
endcoded['title'] = title_labels
endcoded['fuel'] = fuel_labels
endcoded['title_status'] = title_labels


In [ ]:
endcoded.notna()